In [ ]:
import numpy as np
from cirq import (
    CCNOT,
    CNOT,
    CZ,
    ISWAP,
    I,
    X,
    Y,
    Z,
    H,
    SWAP,
    Gate,
    LineQubit,
    Circuit,
    depolarize,
    ops,
    DepolarizingChannel,
    AsymmetricDepolarizingChannel,
    unitary,
)

from mitiq.pec.channels import _operation_to_choi, _circuit_to_choi

from mitiq.pec.representations.biased_noise import (
    represent_operation_with_biased_noise,
)

Test defined in `test_biased_noise_representation_with_choi` is failing for non-zero noise strength `epsilon`. I isolated the function here and added an option to select the noise channel, checking whether the `MixedUnitary` channel behaves as expected, i.e. like a `DepolarizingChannel` when noise bias `eta` = 0.


In [ ]:
def test_biased_noise_representation_with_choi(
    gate: Gate, epsilon: float, eta: float, option
 ):
    """Tests the representation by comparing exact Choi matrices."""
    qreg = LineQubit.range(gate.num_qubits())
    ideal_choi = _operation_to_choi(gate.on(*qreg))
    op_rep = represent_operation_with_biased_noise(
        Circuit(gate.on(*qreg)), epsilon, eta
    )
    choi_components = []
    eta1 = 1 - epsilon
    eta2 = epsilon / (3 * (eta + 1))
    eta3 = epsilon * (3 * eta + 1) / (3 * (eta + 1))

    mix = [
        (eta1, unitary(I)),
        (eta2, unitary(X)),
        (eta2, unitary(Y)),
        (eta3, unitary(Z)),
    ]

    for noisy_op, coeff in op_rep.basis_expansion.items():
        implementable_circ = noisy_op.circuit()
        # The representation assume local noise on each qubit.
        depolarizing_op = DepolarizingChannel(epsilon).on_each(*qreg)
        if option == "mixed":
            biased_op = ops.MixedUnitaryChannel(mix).on_each(*qreg)
        elif option == "local":
            biased_op = DepolarizingChannel(epsilon).on_each(*qreg)
        elif option == "global":
            biased_op = DepolarizingChannel(epsilon, len(qreg))(*qreg)
        elif option == "asymmetric":
            biased_op = AsymmetricDepolarizingChannel(eta2, eta2, eta3).on_each(*qreg)
        # Apply noise after each sequence.
        # NOTE: noise is not applied after each operation.
        implementable_circ.append(biased_op)
        sequence_choi = _circuit_to_choi(implementable_circ)
        choi_components.append(coeff * sequence_choi)
    combination_choi = np.sum(choi_components, axis=0)
    return (ideal_choi - combination_choi)

All of the entries of the following matrices should be < 10**-6 to pass the test defined in `test_biased_noise_representation_with_choi`

In [ ]:
choi_comparison1 = test_biased_noise_representation_with_choi(gate=CZ, epsilon=0.1, eta=0, option="mixed")
print(choi_comparison1)
choi_comparison2 = test_biased_noise_representation_with_choi(gate=CZ, epsilon=0.1, eta=0, option="local")
print(choi_comparison2)
choi_comparison3 = test_biased_noise_representation_with_choi(gate=CZ, epsilon=0.1, eta=0, option="global")
print(choi_comparison3)

Also compare `MixedUnitary` channel with the `AsymmetricDepolarizing` channel, with `eta` = 1

In [ ]:
choi_comparison4 = test_biased_noise_representation_with_choi(gate=CZ, epsilon=0.1, eta=1, option="mixed")
print(choi_comparison4)

choi_comparison5 = test_biased_noise_representation_with_choi(gate=CZ, epsilon=0.1, eta=1, option="asymmetric")
print(choi_comparison5)